In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Package Installation and Import

In [ ]:
!pip install  dgl -f https://data.dgl.ai/wheels/cu118/repo.html
!pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html
!pip install --upgrade numpy
!pip install hypernetx
!pip install cdlib
!pip install torch_geometric

In [ ]:
import dgl
import copy
import bisect
from scipy.sparse import csr_matrix
import json
import networkx as nx
import networkx.algorithms.community as nx_comm
import torch
import numpy as np
from scipy.sparse import coo_matrix
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from networkx.readwrite import json_graph
from numpy import linalg as LA
import random
from gensim.models import Word2Vec
import scipy
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, recall_score,precision_score, accuracy_score,average_precision_score,precision_recall_curve,auc
from torch.nn import Parameter
import torch.optim as optim
import pdb
import itertools
from cdlib import algorithms
from torch_geometric.datasets import Planetoid
from collections import defaultdict
from torch_geometric.datasets import CitationFull

Note: to be able to use all crisp methods, you need to install some additional packages:  {'bayanpy', 'wurlitzer', 'leidenalg', 'graph_tool', 'infomap'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'pyclustering', 'ASLPAw'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'leidenalg', 'wurlitzer', 'infomap'}


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Dataset Read and coms_G creation

In [ ]:
from torch_geometric.utils import to_networkx
path = '/content/drive/MyDrive/My Drive/Colab Notebooks/Current Project/A generalized HT/data'
dataset = Planetoid(path, name='cora')#Please use 'citeseer', 'pubmed' if you want to run on them
#dataset = CitationFull(path,name='dblp')#If you want to run on dblp dataset, just uncomment this line
data = dataset[0]
default_feat = data.x
label = data.y
numpy_array = label.numpy()
label = numpy_array.tolist()
number_class=dataset.num_classes

# Get edge_index and edge_weight as numpy arrays
edge_index = data.edge_index.numpy()
G = nx.Graph()

# Add edges along with default weight of 1
for i in range(edge_index.shape[1]):
    src, dst = edge_index[:, i]
    G.add_edge(src, dst, weight=1)
#convert networkx to dgl
g = dgl.from_networkx(G)

In [ ]:
coms = algorithms.wCommunity(G,min_bel_degree=0.3,threshold_bel_degree=0.3)

# Adding global nodes

In [ ]:
c=coms.communities
frozen_coms_G=[]
coms_G=[]
for i in range (len(c)):
  frozen_coms_G.append(G.subgraph(c[i]))
for i in range (len(frozen_coms_G)):
  coms_G.append(nx.Graph(frozen_coms_G[i]))


centrality = nx.closeness_centrality(G)
newA = set(sorted(centrality, key=centrality.get, reverse=True)[:3])


for k in range(len(coms_G)):
  coms_G_nodes=list( coms_G[k].nodes())
  for i in coms_G_nodes:
    for j in newA:
      if i!=j:
        coms_G[k].add_edge(i,j)


DICT = dict()
for i in range (len(coms_G)):
  DICT[i]=list(coms_G[i].nodes)

# HyG

In [ ]:
nl=coo_matrix((G.number_of_nodes(), G.number_of_nodes()))
nl.setdiag(1)
#nl.toarray()
values = nl.data
indices = np.vstack((nl.row, nl.col))
i = torch.LongTensor(indices)
v = torch.FloatTensor(values)
shape = nl.shape
v_feat=torch.sparse_coo_tensor(i, v, torch.Size(shape))

In [ ]:
n_nodes = G.number_of_nodes()
LEN=len(DICT)
n_hedge = LEN

member_citing = []
for community in DICT.keys():
    members = DICT[community]
    for member in members :
        member_citing.append([member, community])

member_community = torch.LongTensor(member_citing)
data_dict = {
        ('node', 'in', 'edge'): (member_community[:,0], member_community[:,1]),
        ('edge', 'con', 'node'): (member_community[:,1], member_community[:,0])
    }

lst=[]
for i in member_citing:
  lst.append(i[0])
s=set(lst)
s=len(s)
num_nodes_dict = {'edge': LEN,'node':s}
hyG = dgl.heterograph(data_dict,num_nodes_dict=num_nodes_dict)
rows=n_nodes
columns=n_hedge


len_rows=rows
nl=np.eye(len_rows)
nl=torch.from_numpy(nl)
v_feat=nl

len_edges=n_hedge
nl=np.eye(len_edges)
nl=torch.from_numpy(nl)
e_feat=nl


'''
e_feat=coo_matrix((columns, 16))
e_feat.setdiag(1)
#nl.toarray()
values = e_feat.data
indices = np.vstack((e_feat.row, e_feat.col))
i = torch.LongTensor(indices)
v = torch.FloatTensor(values)
shape = e_feat.shape
e_feat=torch.sparse_coo_tensor(i, v, torch.Size(shape))
'''

hyG.ndata['h'] = {'edge' : e_feat.type('torch.FloatTensor'), 'node' : v_feat.type('torch.FloatTensor')}
e_feat = e_feat.type('torch.FloatTensor')
v_feat=v_feat.type('torch.FloatTensor')

v_feat=v_feat.to(device)
e_feat=e_feat.to(device)
hyG=hyG.to(device)

# GCN

In [ ]:
g=g.add_self_loop()
from dgl.nn import GraphConv
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, out_feats):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, out_feats)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h
    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()


nl=np.eye(g.num_nodes())
nl=torch.from_numpy(nl)
nl=torch.tensor(nl,dtype=torch.long)
node_feat=default_feat
gcn_model=GCN(node_feat.shape[1],512,LEN)

# k-core

In [ ]:
def k_core(graph):
    graph.remove_edges_from(nx.selfloop_edges(graph))
    k_core_dict = {node: 0 for node in graph.nodes()}

    degrees = dict(graph.degree())
    if not degrees:
        print("Graph has no nodes or edges.")
        return
    else:
        max_k = max(degrees.values())
        nodes_sorted_by_degree = sorted(degrees, key=degrees.get)

    for k in range(1, max_k + 1):
        while nodes_sorted_by_degree and degrees[nodes_sorted_by_degree[0]] < k:
            node = nodes_sorted_by_degree.pop(0)
            k_core_dict[node] = degrees[node]
            for neighbor in graph[node]:
                if degrees[neighbor] > degrees[node]:
                    degrees[neighbor] -= 1
                    position = nodes_sorted_by_degree.index(neighbor)
                    nodes_sorted_by_degree.pop(position)
                    bisect.insort(nodes_sorted_by_degree, neighbor, lo=position)
            del degrees[node]
            graph.remove_node(node)

    max_value = sum(k_core_dict.values())
    if max_value == 0:
        return k_core_dict
    else:
        normalized_dict = {key: value / max_value for key, value in k_core_dict.items()}
        return normalized_dict

# Laplacian eigenvectors

In [ ]:
building_edges=[]
member_nodes=[]
for i in DICT.keys():
  for j in DICT[i]:
    j=str(j)+'a'
    building_edges.append((j,i))
    member_nodes.append(j)
member_nodes=set(member_nodes)


B = nx.Graph()
B.add_edges_from(building_edges)

node=member_nodes
from networkx.algorithms import bipartite
G_B = bipartite.weighted_projected_graph(B, node)


A=nx.to_numpy_array(G_B)
degree=[]
for i in range (len(A)):
  c=0
  for j in range (len(A)):
    c=c+A[i][j]
  degree.append(c)
degree=np.array(degree)
D=np.diag(degree)
L=D-A

'''
import scipy.sparse as sp
A = sp.csr_matrix(nx.adjacency_matrix(G_B))
degree = A.sum(axis=1).A1
D = sp.diags(degree)
L = D - A
'''
'''
L = nx.laplacian_matrix(G_B)
L = csr_matrix(L)  # Convert to sparse matrix for efficient computation
L = L.toarray()
'''

'\nL = nx.laplacian_matrix(G_B)\nL = csr_matrix(L)  # Convert to sparse matrix for efficient computation\nL = L.toarray()\n'

In [ ]:
# Compute eigenvectors and eigenvalues
eign_v, eign_vec = LA.eigh(L)

# Perform random sign flipping
for i in range(eign_vec.shape[1]):
    maximum_absolute_value_index = np.argmax(np.abs(eign_vec[:, i]))
    sign = np.sign(eign_vec[maximum_absolute_value_index, i])
    random_signs = np.random.choice([-1, 1], size=eign_vec.shape[0])
    eign_vec[:, i] *= sign * random_signs

# Convert eigenvectors and eigenvalues to torch tensors
eign_vec = torch.tensor(eign_vec)
eign_vec = eign_vec.type(torch.FloatTensor).to(device)

eign_v = torch.tensor(eign_v)
eign_v = eign_v.type(torch.FloatTensor).to(device)

# HyGNN

In [ ]:
centrality_values=list(nx.closeness_centrality(G).values())
centrality_values = torch.LongTensor(centrality_values).to(device)

In [ ]:
# Find the number of keys each value appears in
value_counts = defaultdict(int)
for key, values in DICT.items():
    unique_values = set(values)
    for value in unique_values:
        value_counts[value] += 1

# Calculate the score for each value
total_keys = len(DICT)
value_scores = {}
uniqueness_list=[]
for value, count in value_counts.items():
    score = 1 - (count / total_keys)
    value_scores[value] = score
    uniqueness_list.append(score)

uniqueness = np.array(uniqueness_list)
uniqueness = torch.LongTensor(uniqueness).to(device)

In [ ]:
def local_clustering_coefficient(com_graph):
    clustering_coefficients = {}
    hyperedge_nodes = list(com_graph.nodes())
    for node in hyperedge_nodes:
        neighbors_in_hyperedge = set(com_graph.neighbors(node))
        n = len(neighbors_in_hyperedge)
        if n > 1:
            # Calculate the number of existing connections between neighbors
            existing_connections = sum(1 for u, v in itertools.combinations(neighbors_in_hyperedge, 2) if com_graph.has_edge(u, v))
            total_possible_connections = n * (n - 1) // 2
            clustering_coefficients[node] = existing_connections / total_possible_connections
        else:
            clustering_coefficients[node] = 0

    return clustering_coefficients


def hyperedge_clustering_coefficient(com_graph):
    hyperedge_nodes = list(com_graph.nodes())
    triangle_count = 0

    for idx, u in enumerate(hyperedge_nodes):
        for v in hyperedge_nodes[idx + 1:]:
            if com_graph.has_edge(u, v):  # Ensure there is an edge between u and v
                # common_neighbors = list(nx.common_neighbors(com_graph, u, v))
                # for w in common_neighbors:
                #     if com_graph.has_edge(u, w) and com_graph.has_edge(v, w):  # Check if u, v, w form a triangle
                        triangle_count += len(list(nx.common_neighbors(com_graph, u, v)))
                        # triangle_count += 1

    # Since each triangle is counted three times, divide by 3
    triangle_count //= 3
    n = len(hyperedge_nodes)
    total_possible_triangles = n * (n - 1) * (n - 2) // 6

    clustering_coefficient = triangle_count / total_possible_triangles if total_possible_triangles > 0 else 0.0
    return clustering_coefficient

size_of_coms_G=len(coms_G)
dict_hyperedge_clustering_coefficient={}
for i in range (size_of_coms_G):
  dict_hyperedge_clustering_coefficient[i]=hyperedge_clustering_coefficient(coms_G[i])

In [ ]:
num_of_edge=int(hyG.num_edges()/2)

class SAT_HG(nn.Module):
    # edge attention  version
    def __init__(self, input_dim, query_dim, vertex_dim, edge_dim, num_class, dropout):
        super(SAT_HG, self).__init__()

        self.query_dim = query_dim
        self.vtx_lin_1layer = torch.nn.Linear(input_dim, vertex_dim)
        self.vtx_lin = torch.nn.Linear(vertex_dim, vertex_dim)

        self.qe_lin = torch.nn.Linear(edge_dim, query_dim)
        self.kv_lin = torch.nn.Linear(vertex_dim, query_dim)
        self.vv_lin = torch.nn.Linear(vertex_dim, edge_dim)

        self.qv_lin = torch.nn.Linear(vertex_dim, query_dim)
        self.ke_lin = torch.nn.Linear(edge_dim, query_dim)
        self.ve_lin = torch.nn.Linear(edge_dim, vertex_dim)


        self.cls = nn.Linear(vertex_dim, num_class)
        self.cs_embedding = nn.Embedding(num_embeddings=len(centrality_values), embedding_dim=LEN)
        self.un_embedding = nn.Embedding(num_embeddings=len(uniqueness), embedding_dim=LEN)

        self.eign_vec_lin=torch.nn.Linear(eign_vec.shape[1],vertex_dim)
        self.edge_lin_1layer=torch.nn.Linear(e_feat.shape[0],e_feat.shape[1])


        #add&norm
        self.layer_norm1 = nn.LayerNorm(vertex_dim)
        self.layer_norm2 = nn.LayerNorm(vertex_dim)

        #ffn
        self.linear1 = nn.Linear(vertex_dim, query_dim)
        self.linear3 = nn.Linear(vertex_dim, query_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(query_dim, vertex_dim)
        self.linear4 = nn.Linear(query_dim, vertex_dim)


    def add_and_norm1(self, x, residual):
        output = x + residual
        output = self.layer_norm1(output)
        return output

    def ffn1(self, x):
        output = self.linear1(x)
        output = self.relu(output)
        output = self.dropout(output)
        output = self.linear2(output)
        return output

    def add_and_norm2(self, x, residual):
        output = x + residual
        output = self.layer_norm2(output)
        return output

    def ffn2(self, x):
        output = self.linear3(x)
        output = self.relu(output)
        output = self.dropout(output)
        output = self.linear4(output)
        return output

    def attention(self, edges):
        attn_score = F.leaky_relu((edges.src['k'] * edges.dst['q']).sum(-1))
        c=attn_score/np.sqrt(self.query_dim)
        return {'Attn': c}

    def message_func(self, edges):
        return {'v': edges.src['v'], 'Attn': edges.data['Attn']}

    def reduce_func1(self, nodes):
        #attention_score = F.softmax((nodes.mailbox['Attn']), dim=1)
        #aggr = torch.sum(attention_score.unsqueeze(-1) * nodes.mailbox['v'], dim=1)


        lst_node=nodes.nodes().tolist()
        updated_attention_score_list=[]
        node_embedding_gcn_list=[]
        for i in range (len(lst_node)):
          node=lst_node[i]
          attn_score=nodes.mailbox['Attn'][i]

          # Calculate clustering coefficients for nodes within the hyperedge
          com_graph=nx.Graph()
          com_graph=copy.deepcopy(coms_G[node])

          clustering_coefficients = local_clustering_coefficient(coms_G[node])
          clustering_coefficients=torch.Tensor(list(clustering_coefficients.values())).to(device)

          k_core_values=list(k_core(com_graph).values())
          k_core_values=torch.Tensor(k_core_values).to(device)

          combined_local_structural_values=k_core_values+clustering_coefficients

          updated_attention_score=torch.add(attn_score,combined_local_structural_values)
          updated_attention_score=updated_attention_score.tolist()
          updated_attention_score_list.append(updated_attention_score)

        updated_attention_score_list=torch.Tensor(updated_attention_score_list).to(device)


        updated_attention_score_list = F.softmax(updated_attention_score_list, dim=1)
        aggr = torch.sum(updated_attention_score_list.unsqueeze(-1) *nodes.mailbox['v'], dim=1)

        return {'h': aggr}

    def reduce_func2(self, nodes):
        lst_node=nodes.nodes().tolist()

        #attention_score = F.softmax((nodes.mailbox['Attn']), dim=1)
        #aggr = torch.sum(attention_score.unsqueeze(-1) * nodes.mailbox['v'], dim=1)

        updated_attention_score_list=[]
        for i in range (len(lst_node)):
          node=lst_node[i]
          attn_score=nodes.mailbox['Attn'][i]

          key_=[key for key,value in DICT.items() if node in value]
          combined_global_structural_values=[]

          for k in key_:

            #c=hyperedge_clustering_coefficient(coms_G[k])
            c=dict_hyperedge_clustering_coefficient[k]
            d=c+(coms_G[k].number_of_nodes()/G.number_of_nodes())
            combined_global_structural_values.append(d)

          combined_global_structural_values=torch.Tensor(combined_global_structural_values).to(device)
          updated_attention_score=torch.add(attn_score,combined_global_structural_values)
          updated_attention_score=updated_attention_score.tolist()
          updated_attention_score_list.append(updated_attention_score)

        updated_attention_score_list=torch.Tensor(updated_attention_score_list).to(device)
        updated_attention_score_list = F.softmax(updated_attention_score_list, dim=1)

        aggr = torch.sum(updated_attention_score_list.unsqueeze(-1) * nodes.mailbox['v'], dim=1)


        return {'h': aggr}

    def forward(self, hyG, vfeat, efeat, first_layer, last_layer):

        with hyG.local_scope():

            if first_layer:
                feat_v = self.vtx_lin_1layer(vfeat)
                pe=self.eign_vec_lin(eign_vec)
                cs=self.cs_embedding(centrality_values)
                un=self.un_embedding(uniqueness)


                feat_v=feat_v+pe
                feat_v_gcn=gcn_model(g,node_feat).to(device)
                feat_v=feat_v+feat_v_gcn
                feat_v=feat_v+cs
                feat_v=feat_v+un
            else:
                feat_v = self.vtx_lin(vfeat)
                pe=self.eign_vec_lin(eign_vec)
                cs=self.cs_embedding(centrality_values)
                un=self.un_embedding(uniqueness)


                feat_v=feat_v+pe
                feat_v_gcn=gcn_model(g,node_feat).to(device)
                feat_v=feat_v+feat_v_gcn
                feat_v=feat_v+cs
                feat_v=feat_v+un

            feat_e = efeat

            # node attention
            hyG.ndata['h'] = {'node': feat_v}
            hyG.ndata['k'] = {'node' : self.kv_lin(feat_v)}
            hyG.ndata['v'] = {'node' : self.vv_lin(feat_v)}
            hyG.ndata['q'] = {'edge' : self.qe_lin(feat_e)}
            hyG.apply_edges(self.attention, etype='in')
            hyG.update_all(self.message_func, self.reduce_func1, etype='in')
            feat_e_transformed = hyG.ndata['h']['edge']

            feat_e_add_norm=self.add_and_norm1(feat_e_transformed,feat_e)
            feat_e_ffn=self.ffn1(feat_e_add_norm)
            feat_e=self.add_and_norm1(feat_e_ffn,feat_e_add_norm)

            # edge attention
            hyG.ndata['k'] = {'edge' : self.ke_lin(feat_e)}
            hyG.ndata['v'] = {'edge' : self.ve_lin(feat_e)}
            hyG.ndata['q'] = {'node' : self.qv_lin(feat_v)}
            hyG.apply_edges(self.attention, etype='con')
            hyG.update_all(self.message_func, self.reduce_func2, etype='con')
            feat_v_transformed = hyG.ndata['h']['node']


            feat_v_add_norm=self.add_and_norm2(feat_v_transformed,feat_v)
            feat_v_ffn=self.ffn2(feat_v_add_norm)
            feat_v=self.add_and_norm2(feat_v_ffn,feat_v_add_norm)

            if not last_layer :
                feat_v = F.dropout(feat_v, self.dropout)
                return feat_v
            if last_layer:
                feat_v = self.dropout(feat_v)
                pred=self.cls(feat_v)
                #return feat_v, feat_e, pred
                return pred
            else:
                return [g, feat_v, feat_e]


# Train-test

In [ ]:
'''
Don't run second time
'''
with open('/content/drive/MyDrive/My Drive/Colab Notebooks/Current Project/A generalized HT/data/label_cora.txt', 'w') as f:
    for item in label:
        f.write("%s \n" % item)

In [ ]:
label_data=pd.read_csv('/content/drive/MyDrive/My Drive/Colab Notebooks/Current Project/A generalized HT/data/label_cora.txt',header=None)
label=label_data[0].tolist()

test_size=0.25

train_label, test_label= train_test_split(label,test_size=test_size, random_state=7)
train_label=torch.tensor(train_label,dtype=torch.long).to(device)
size=int(len(label)*0.25)
val_label=train_label[0:size]
train_label=train_label[size:]
test_label=torch.tensor(test_label,dtype=torch.long).to(device)

# Multi-Head

In [ ]:
class Multi_head_attn(nn.Module):

    def __init__(self, input_dim, query_dim, vertex_dim, edge_dim, number_class, dropout,num_heads,merge='cat'):
        super(Multi_head_attn, self).__init__()

        self.heads = nn.ModuleList()
        for i in range(num_heads):
            self.heads.append(SAT_HG(input_dim, query_dim, vertex_dim, edge_dim, number_class, dropout ))
        self.merge = merge

    def forward(self, hyG, v_feat,e_feat,first_layer,second_layer):
        head_outs = [attn_head(hyG,v_feat,e_feat,first_layer,second_layer) for attn_head in self.heads]

        if self.merge == 'cat':
            # concat on the output feature dimension (dim=1)
            return torch.cat(head_outs, dim=1)
        else:
            # merge using average
            return torch.mean(torch.stack(head_outs))
    def reset_parameters(self):
        self.heads.reset_parameters()
class SAT_HG_attn(nn.Module):
    def __init__(self, input_dim, query_dim, vertex_dim, edge_dim, number_class, dropout, num_heads):
        super(SAT_HG_attn, self).__init__()
        self.layer1 = Multi_head_attn(input_dim, query_dim, vertex_dim, edge_dim, number_class, dropout,num_heads )
    def forward(self,hyG,v_feat,e_feat,first_layer,second_layer):
        x = self.layer1(hyG, v_feat,e_feat,True,True)
        return x

# Train

In [ ]:
model =SAT_HG_attn(v_feat.shape[1], 64, LEN, LEN,  number_class, 0.5,4)
model=model.to(device)
loss_fn = nn.CrossEntropyLoss()

#optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), gcn_model.parameters()), lr=0.01)
best_val_acc = 0
patience=0

for i in range(500):
    model.train()
    pred= model(hyG, v_feat, e_feat, True,True)
    train_pred, test_pred= train_test_split( pred,test_size=test_size, random_state=7)
    val_pred=train_pred[0:size]
    train_pred=train_pred[size:]


    loss = loss_fn(train_pred, train_label)
    pred_cls = torch.argmax(train_pred, -1)
    train_acc = torch.eq(pred_cls, train_label).sum().item()/len(train_label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.no_grad():

      model.eval()
      val_cls = torch.argmax(val_pred, -1)
      val_acc = torch.eq(val_cls, val_label).sum().item()/len(val_label)

      # Save the best validation accuracy and the corresponding test accuracy.
      if best_val_acc < val_acc:
        best_val_acc = val_acc
        E=i
        patience=0
        torch.save(test_pred,'latest.pth')
      else:
        patience+=1
      if patience==100:
        break
    if i % 10 == 0:
      print('In epoch {}, train loss: {:.4f}, val_acc: {:.4f} (best_val_acc: {:.4f})'.format(i, loss, val_acc, best_val_acc))

In epoch 0, train loss: 3.4929, val_acc: 0.0857 (best_val_acc: 0.0857)
In epoch 10, train loss: 0.1207, val_acc: 0.9158 (best_val_acc: 0.9173)
In epoch 20, train loss: 0.0340, val_acc: 0.8833 (best_val_acc: 0.9173)
In epoch 30, train loss: 0.0131, val_acc: 0.8804 (best_val_acc: 0.9173)
In epoch 40, train loss: 0.0027, val_acc: 0.8759 (best_val_acc: 0.9173)
In epoch 50, train loss: 0.0009, val_acc: 0.8744 (best_val_acc: 0.9173)
In epoch 60, train loss: 0.0007, val_acc: 0.8700 (best_val_acc: 0.9173)
In epoch 70, train loss: 0.0005, val_acc: 0.8833 (best_val_acc: 0.9173)
In epoch 80, train loss: 0.0001, val_acc: 0.8744 (best_val_acc: 0.9173)
In epoch 90, train loss: 0.0002, val_acc: 0.8789 (best_val_acc: 0.9173)
In epoch 100, train loss: 0.0001, val_acc: 0.8774 (best_val_acc: 0.9173)


In [ ]:
best_test_perd=torch.load('latest.pth')
with torch.no_grad():
  test_pred=best_test_perd
  test_cls = torch.argmax(test_pred, -1)
  test_acc = torch.eq(test_cls, test_label).sum().item()/len(test_label)
  print(' Best Epoch: {}, Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-score {:.4f}'.format(E,accuracy_score(test_label.cpu(),test_cls.cpu()),precision_score(test_label.cpu(),test_cls.cpu(),average='weighted'),recall_score(test_label.cpu(),test_cls.cpu(),average='weighted'),f1_score(test_label.cpu(),test_cls.cpu(),average='weighted')))

 Best Epoch: 5, Accuracy: 0.9306, Precision: 0.9309, Recall: 0.9306, F1-score 0.9302
